In [ ]:
!pip install -q kaggle torch torchvision torchmetrics ipywidgets

In [ ]:
import gc
import os
import time
import random
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from copy import deepcopy
# from tqdm import tqdm
from tqdm.notebook import tqdm


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchmetrics import Accuracy
import torchvision.models as models

In [ ]:
%matplotlib inline
warnings.filterwarnings("ignore")

Подготовка данных

In [ ]:
ROOT_DIR = './'
SPLIT_DATA_DIR = ROOT_DIR + 'data/'

TRAIN_DATA_DIR = SPLIT_DATA_DIR + 'train/'
VALID_DATA_DIR = SPLIT_DATA_DIR + 'valid/'
TEST_DATA_DIR = SPLIT_DATA_DIR + 'test/'

In [ ]:
def train_valid_split(
    num_val_images_per_class,
    num_test_images_per_class
):

    os.makedirs(VALID_DATA_DIR, exist_ok = True)

    classes = os.listdir(TRAIN_DATA_DIR)

    for class_name in classes:

        list_of_pics = os.listdir(TRAIN_DATA_DIR + class_name)

        os.mkdir(VALID_DATA_DIR + class_name)

        val_list_of_pics = np.random.choice(list_of_pics, size = num_val_images_per_class, replace=False)

        for pic in val_list_of_pics:
            shutil.move(TRAIN_DATA_DIR + class_name + '/' + pic, VALID_DATA_DIR + class_name + '/' + pic)

    os.makedirs(TEST_DATA_DIR, exist_ok = True)

    for class_name in classes:

        list_of_pics = os.listdir(TRAIN_DATA_DIR + class_name)

        os.mkdir(TEST_DATA_DIR + class_name)

        val_list_of_pics = np.random.choice(list_of_pics, size = num_val_images_per_class, replace=False)

        for pic in val_list_of_pics:
            shutil.move(TRAIN_DATA_DIR + class_name + '/' + pic, TEST_DATA_DIR + class_name + '/' + pic)

if not os.path.isdir(VALID_DATA_DIR):
    train_valid_split(100, 50)

In [ ]:
torch.manual_seed(42)
BATCH_SIZE = 64
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(p=0.5)
])

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(TRAIN_DATA_DIR, transform=transform_train)
valid_dataset = datasets.ImageFolder(VALID_DATA_DIR, transform=transform_valid)
test_dataset = datasets.ImageFolder(TEST_DATA_DIR, transform = transform_test)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True
)

valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = 1,
    shuffle = False
)

**Автоэнкодер**

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        def initialization(layer):
            if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.ConvTranspose2d):
                nn.init.kaiming_normal_(layer.weight)
                layer.bias.data.fill_(0.)

        self.encoder = nn.Sequential(nn.Conv2d(3, 128, kernel_size=(3, 3), padding = 1, stride = 1),
                                     nn.ELU(),
                                     nn.MaxPool2d(kernel_size=(2, 2)),
                                     nn.Conv2d(128, 64, kernel_size=(3, 3), padding = 1, stride = 1),
                                     nn.ELU(),
                                     nn.MaxPool2d(kernel_size=(2, 2)),
                                     nn.Conv2d(64, 32, kernel_size=(3, 3), padding = 1, stride = 1),
                                     nn.ELU()
        )

        self.decoder = nn.Sequential(nn.ConvTranspose2d(32, 64, kernel_size=(3, 3),
                                                        padding = 1, stride = 1),
                                     nn.ELU(),
                                     nn.Upsample(scale_factor=2, mode='nearest'),
                                     nn.ConvTranspose2d(64, 128, kernel_size=(3, 3),
                                                        padding = 1, stride = 1),
                                     nn.ELU(),
                                     nn.Upsample(scale_factor=2, mode='nearest'),
                                     nn.ConvTranspose2d(128, 3, kernel_size=(3, 3),
                                                        padding = 1, stride = 1),
                                     nn.ELU()
        )

        self.apply(initialization)

    def forward(self, x):
        latent_code = self.encoder(x)
        reconstruction = self.decoder(latent_code)
        return reconstruction

In [ ]:
for x_train, y_train in  train_dataloader:
    break

for x_valid, y_valid in  valid_dataloader:
    break

In [ ]:
model = AutoEncoder()
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0003)

def run_train(model, criterion, optimizer, x_train, x_valid):
    for epoch in range(50):
        print(f'epoch: {epoch}')

        model.train()
        optimizer.zero_grad()
        output = model(x_train)
        loss = criterion(output, x_train)
        loss.backward()
        print(f'loss_train: {loss.item()}')
        optimizer.step()

        model.eval()
        output = model(x_valid)
        loss = criterion(output, x_valid)
        print(f'loss_valid: {loss.item()}')

run_train(model, criterion, optimizer, x_train, x_valid) # переобучение на одном батче


In [ ]:
class FullModel(nn.Module):
    def __init__(self,ae_model, num_classes: int = 200) -> None:
        super(FullModel, self).__init__()

        self.features = nn.Sequential(
            ae_model.encoder
        )

        self.avgpool = nn.AvgPool2d((3, 3))

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(32 * 5 * 5, 2024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2024, 2024),
            nn.ReLU(),
            nn.Linear(2024, num_classes),
        )

    def _forward_impl(self, x):
        out = self.features(x)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)

        return out

    def forward(self, x):
        return self._forward_impl(x)

In [ ]:
class model_training():

    def __init__(self, lr, train_autoencode, trainloader, testloader, device, model):
        self.trainloader = trainloader
        self.testloader = testloader
        self.device = device
        self.train_autoencode = train_autoencode

        self.model = model
        if train_autoencode:
            self.loss_fn = torch.nn.MSELoss()
        else:
            self.loss_fn = torch.nn.CrossEntropyLoss()


        self.opt = optim.Adam(
            model.parameters(),
            lr = lr,
            weight_decay = 0.0001
        )

        self.best_model = None
        self.best_epoch = None

        self.loss_train = []
        self.loss_test = []
        self.metric_train = []
        self.metric_test = []

    def accuracy(self, y_predicts, y_labels):
        acc = (y_predicts == y_labels).sum().item() / y_predicts.size(0)
        return acc

    def train_nn(self, trainloader, model, opt, loss_fn, device):

        model.train()
        running_loss_train = []
        running_acc_train = []
        if self.train_autoencode:
            for batch in tqdm(train_dataloader):
                inputs, labels = batch
                inputs = inputs.to(device)

                opt.zero_grad()
                outputs = model(inputs).to(device)


                loss = loss_fn(outputs, inputs)
                loss.backward()
                opt.step()

                running_loss_train.append(loss.item())
                running_acc_train.append(loss.item())
        else:
            for batch in tqdm(train_dataloader):
                inputs, labels = batch
                inputs = inputs.to(device)
                labels = labels.to(device)

                opt.zero_grad()
                outputs = model(inputs).to(device)
                y_pred = nn.Softmax(dim=1)(outputs).argmax(dim=1).to(device)

                loss = loss_fn(outputs, labels)
                loss.backward()
                opt.step()

                running_loss_train.append(loss.item())
                running_acc_train.append(self.accuracy(y_pred, labels))


        return model, running_loss_train, running_acc_train

    def eval_nn(self, testloader, model, loss_fn, device):

        model.eval()
        running_loss_test = []
        running_acc_test = []
        if self.train_autoencode:
            for batch in tqdm(valid_dataloader):
                inputs, labels = batch
                inputs = inputs.to(device)

                outputs = model(inputs).to(device)
                loss = loss_fn(outputs, inputs)

                running_loss_test.append(loss.item())
                running_acc_test.append(loss.item())
        else:


            for batch in tqdm(valid_dataloader):

                inputs, labels = batch
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs).to(device)
                y_pred = nn.Softmax(dim=1)(outputs).argmax(dim=1).to(device)
                loss = loss_fn(outputs, labels)

                running_loss_test.append(loss.item())
                running_acc_test.append(self.accuracy(y_pred, labels))

        return running_loss_test, running_acc_test

    def training_loop(self, max_epochs = 10):

        print('Начинаю обучение')
        start_training_time = time.time()

        for epoch in tqdm(range(max_epochs)):

            start_epoch_time = time.time()

            self.model, running_loss_train, running_acc_train = self.train_nn(
                self.trainloader,
                self.model,
                self.opt,
                self.loss_fn,
                self.device
            )

            running_loss_test, running_acc_test = self.eval_nn(
                self.testloader,
                self.model,
                self.loss_fn,
                self.device
            )

            self.loss_train.append(np.mean(running_loss_train))
            self.loss_test.append(np.mean(running_loss_test))

            self.metric_train.append(np.mean(running_acc_train))
            self.metric_test.append(np.mean(running_acc_test))

            if np.mean(running_acc_test) >= np.max(self.metric_test):
                self.best_model = deepcopy(self.model)
                self.best_epoch = epoch
                torch.save(main_loop.best_model, './models/custom_model.pt')

            duration_epoch = time.time() - start_epoch_time

            print(f"""EPOCH {epoch} :
            train_loss: {self.loss_train[-1]:.5f}
            test_loss: {self.loss_test[-1]:.5f}
            train_acc: {self.metric_train[-1]:.5f}
            test_acc: {self.metric_test[-1]:.5f}
            Эпоха заняла по времени {round(duration_epoch / 60, 2)} минут""")

        duration_total = time.time() - start_training_time
        print(f'Всего обучение заняло: {round(duration_total / 60, 2)} минут')
        print(f'Лучшее значение метрики было достингнуто на {self.best_epoch} эпохе')

        fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize=(10, 8))

        axes[0, 0].plot(range(max_epochs), self.loss_train)
        axes[0, 0].set_title('loss_train')

        axes[0, 1].plot(range(max_epochs), self.loss_test)
        axes[0, 1].set_title('loss_test')

        axes[1, 0].plot(range(max_epochs), self.metric_train)
        axes[1, 0].set_title('metric_train')

        axes[1, 1].plot(range(max_epochs), self.metric_test)
        axes[1, 1].set_title('metric_test')

        plt.tight_layout()
        plt.show()

In [ ]:
model_ae = AutoEncoder()
model_ae.to(DEVICE)
train_autoencode = True

main_loop_ae = model_training(0.01,train_autoencode, train_dataloader, valid_dataloader, DEVICE, model_ae)
main_loop_ae.training_loop(1)

In [ ]:
model = FullModel(model_ae)
model.features.requires_grad_ = False
model.to(DEVICE)
train_autoencode = False
main_loop = model_training(0.01, train_autoencode,
                           train_dataloader,
                           valid_dataloader,
                           DEVICE, model)
main_loop.training_loop(1)

In [ ]:
best_model = torch.load('./models/custom_model.pt')

loss_fn = nn.CrossEntropyLoss()

def accuracy(y_predicts, y_labels):
    acc = (y_predicts == y_labels).sum().item() / y_predicts.size(0)
    return acc

best_model.eval()
running_loss_test = []
running_acc_test = []

for batch in tqdm(valid_dataloader):

    inputs, labels = batch
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)

    outputs = best_model(inputs).to(DEVICE)
    y_pred = nn.Softmax(dim=1)(outputs).argmax(dim=1).to(DEVICE)
    loss = loss_fn(outputs, labels)

    running_loss_test.append(loss.item())
    running_acc_test.append(accuracy(y_pred, labels))


print(np.mean(running_loss_test))
print(np.mean(running_acc_test))

**ResNet**

In [ ]:
resnet18 = models.resnet18(pretrained=True)
print(resnet18)

In [ ]:
class Pretrainde_RESNET18(nn.Module):
    def __init__(self):
        super().__init__()

        pretrained_net = models.resnet18(pretrained=True)

        self.features = torch.nn.Sequential(*(list(pretrained_net.children())[:-1]))

        self.classifier = torch.nn.Sequential(
            nn.Linear(in_features=512, out_features=300),
            nn.Dropout(0.5),
            nn.Linear(in_features=300, out_features=200)
        )
        nn.init.kaiming_normal_(self.classifier[0].weight)
        nn.init.kaiming_normal_(self.classifier[2].weight)

    def forward(self, x):
        features = self.features(x)
        features = nn.Flatten(start_dim = 1)(features)
        logits = self.classifier(features)
        return logits

In [ ]:
class model_training():

    def __init__(self, lr, trainloader, testloader, device, model):
        self.trainloader = trainloader
        self.testloader = testloader
        self.device = device

        self.model = model

        self.loss_fn = nn.CrossEntropyLoss()

        self.opt = optim.Adam(
            model.parameters(),
            lr = lr,
            weight_decay = 0.0001
        )

        self.best_model = None
        self.best_epoch = None

        self.loss_train = []
        self.loss_test = []
        self.metric_train = []
        self.metric_test = []

    def accuracy(self, y_predicts, y_labels):
        acc = (y_predicts == y_labels).sum().item() / y_predicts.size(0)
        return acc

    def train_nn(self, trainloader, model, opt, loss_fn, device):

        model.train()
        running_loss_train = []
        running_acc_train = []

        for batch in tqdm(train_dataloader):
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            opt.zero_grad()
            outputs = model(inputs).to(device)
            y_pred = nn.Softmax(dim=1)(outputs).argmax(dim=1).to(device)

            loss = loss_fn(outputs, labels)
            loss.backward()
            opt.step()

            running_loss_train.append(loss.item())
            running_acc_train.append(self.accuracy(y_pred, labels))

        return model, running_loss_train, running_acc_train

    def eval_nn(self, testloader, model, loss_fn, device):

        model.eval()
        running_loss_test = []
        running_acc_test = []

        for batch in tqdm(valid_dataloader):

            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs).to(device)
            y_pred = nn.Softmax(dim=1)(outputs).argmax(dim=1).to(device)
            loss = loss_fn(outputs, labels)

            running_loss_test.append(loss.item())
            running_acc_test.append(self.accuracy(y_pred, labels))

        return running_loss_test, running_acc_test

    def training_loop(self, max_epochs = 10):

        print('Начинаю обучение')
        start_training_time = time.time()

        for epoch in tqdm(range(max_epochs)):

            start_epoch_time = time.time()

            self.model, running_loss_train, running_acc_train = self.train_nn(
                self.trainloader,
                self.model,
                self.opt,
                self.loss_fn,
                self.device
            )

            running_loss_test, running_acc_test = self.eval_nn(
                self.testloader,
                self.model,
                self.loss_fn,
                self.device
            )

            self.loss_train.append(np.mean(running_loss_train))
            self.loss_test.append(np.mean(running_loss_test))

            self.metric_train.append(np.mean(running_acc_train))
            self.metric_test.append(np.mean(running_acc_test))

            if np.mean(running_acc_test) >= np.max(self.metric_test):
                self.best_model = deepcopy(self.model)
                self.best_epoch = epoch
                torch.save(main_loop.best_model, './models/custom_transfer_model.pt')

            duration_epoch = time.time() - start_epoch_time

            print(f"""EPOCH {epoch} :
            train_loss: {self.loss_train[-1]:.5f}
            test_loss: {self.loss_test[-1]:.5f}
            train_acc: {self.metric_train[-1]:.5f}
            test_acc: {self.metric_test[-1]:.5f}
            Эпоха заняла по времени {round(duration_epoch / 60, 2)} минут""")

        duration_total = time.time() - start_training_time
        print(f'Всего обучение заняло: {round(duration_total / 60, 2)} минут')
        print(f'Лучшее значение метрики было достингнуто на {self.best_epoch} эпохе')

        fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize=(10, 8))

        axes[0, 0].plot(range(max_epochs), self.loss_train)
        axes[0, 0].set_title('loss_train')

        axes[0, 1].plot(range(max_epochs), self.loss_test)
        axes[0, 1].set_title('loss_test')

        axes[1, 0].plot(range(max_epochs), self.metric_train)
        axes[1, 0].set_title('metric_train')

        axes[1, 1].plot(range(max_epochs), self.metric_test)
        axes[1, 1].set_title('metric_test')

        plt.tight_layout()
        plt.show()

In [ ]:
model = Pretrainde_RESNET18()
model.features.requires_grad_ = False
model.to(DEVICE)

main_loop = model_training(0.01, train_dataloader, valid_dataloader, DEVICE, model)
main_loop.training_loop(1)

In [ ]:
best_model = torch.load('./models/custom_transfer_model.pt')

loss_fn = nn.CrossEntropyLoss()

def accuracy(y_predicts, y_labels):
    acc = (y_predicts == y_labels).sum().item() / y_predicts.size(0)
    return acc

best_model.eval()
running_loss_test = []
running_acc_test = []

for batch in tqdm(valid_dataloader):

    inputs, labels = batch
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)

    outputs = best_model(inputs).to(DEVICE)
    y_pred = nn.Softmax(dim=1)(outputs).argmax(dim=1).to(DEVICE)
    loss = loss_fn(outputs, labels)

    running_loss_test.append(loss.item())
    running_acc_test.append(accuracy(y_pred, labels))


print(np.mean(running_loss_test))
print(np.mean(running_acc_test))

**AlexNet**

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 200) -> None:
        super(AlexNet, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, (11, 11), (4, 4), (2, 2)),
            nn.ReLU(),
            nn.MaxPool2d((3, 3), (2, 2)),

            nn.Conv2d(64, 192, (5, 5), (1, 1), (2, 2)),
            nn.ReLU(),
            nn.MaxPool2d((3, 3), (2, 2)),

            nn.Conv2d(192, 384, (3, 3), (1, 1), (1, 1)),
            nn.ReLU(),
            nn.Conv2d(384, 256, (3, 3), (1, 1), (1, 1)),
            nn.ReLU(),
            nn.Conv2d(256, 256, (3, 3), (1, 1), (1, 1)),
            nn.ReLU(),
            nn.MaxPool2d((3, 3), (2, 2)),
        )

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes),
        )

    def _forward_impl(self, x):
        out = self.features(x)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)

        return out

    def forward(self, x):
        return self._forward_impl(x)

In [ ]:
class model_training():

    def __init__(self, lr, trainloader, testloader, device, model):
        self.trainloader = trainloader
        self.testloader = testloader
        self.device = device

        self.model = model

        self.loss_fn = nn.CrossEntropyLoss()

        self.opt = optim.Adam(
            model.parameters(),
            lr = lr,
            weight_decay = 0.0001
        )

        self.best_model = None
        self.best_epoch = None

        self.loss_train = []
        self.loss_test = []
        self.metric_train = []
        self.metric_test = []

    def accuracy(self, y_predicts, y_labels):
        acc = (y_predicts == y_labels).sum().item() / y_predicts.size(0)
        return acc

    def train_nn(self, trainloader, model, opt, loss_fn, device):

        model.train()
        running_loss_train = []
        running_acc_train = []

        for batch in tqdm(train_dataloader):
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            opt.zero_grad()
            outputs = model(inputs).to(device)
            y_pred = nn.Softmax(dim=1)(outputs).argmax(dim=1).to(device)

            loss = loss_fn(outputs, labels)
            loss.backward()
            opt.step()

            running_loss_train.append(loss.item())
            running_acc_train.append(self.accuracy(y_pred, labels))

        return model, running_loss_train, running_acc_train

    def eval_nn(self, testloader, model, loss_fn, device):

        model.eval()
        running_loss_test = []
        running_acc_test = []

        for batch in tqdm(valid_dataloader):

            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs).to(device)
            y_pred = nn.Softmax(dim=1)(outputs).argmax(dim=1).to(device)
            loss = loss_fn(outputs, labels)

            running_loss_test.append(loss.item())
            running_acc_test.append(self.accuracy(y_pred, labels))

        return running_loss_test, running_acc_test

    def training_loop(self, max_epochs = 10):

        print('Начинаю обучение')
        start_training_time = time.time()

        for epoch in tqdm(range(max_epochs)):

            start_epoch_time = time.time()

            self.model, running_loss_train, running_acc_train = self.train_nn(
                self.trainloader,
                self.model,
                self.opt,
                self.loss_fn,
                self.device
            )

            running_loss_test, running_acc_test = self.eval_nn(
                self.testloader,
                self.model,
                self.loss_fn,
                self.device
            )

            self.loss_train.append(np.mean(running_loss_train))
            self.loss_test.append(np.mean(running_loss_test))

            self.metric_train.append(np.mean(running_acc_train))
            self.metric_test.append(np.mean(running_acc_test))

            if np.mean(running_acc_test) >= np.max(self.metric_test):
                self.best_model = deepcopy(self.model)
                self.best_epoch = epoch
                torch.save(main_loop.best_model, './models/custom_model.pt')

            duration_epoch = time.time() - start_epoch_time

            print(f"""EPOCH {epoch} :
            train_loss: {self.loss_train[-1]:.5f}
            test_loss: {self.loss_test[-1]:.5f}
            train_acc: {self.metric_train[-1]:.5f}
            test_acc: {self.metric_test[-1]:.5f}
            Эпоха заняла по времени {round(duration_epoch / 60, 2)} минут""")

        duration_total = time.time() - start_training_time
        print(f'Всего обучение заняло: {round(duration_total / 60, 2)} минут')
        print(f'Лучшее значение метрики было достингнуто на {self.best_epoch} эпохе')

        fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize=(10, 8))

        axes[0, 0].plot(range(max_epochs), self.loss_train)
        axes[0, 0].set_title('loss_train')

        axes[0, 1].plot(range(max_epochs), self.loss_test)
        axes[0, 1].set_title('loss_test')

        axes[1, 0].plot(range(max_epochs), self.metric_train)
        axes[1, 0].set_title('metric_train')

        axes[1, 1].plot(range(max_epochs), self.metric_test)
        axes[1, 1].set_title('metric_test')

        plt.tight_layout()
        plt.show()

In [ ]:
model = AlexNet()
model.to(DEVICE)

main_loop = model_training(0.01, train_dataloader, valid_dataloader, DEVICE, model)
main_loop.training_loop(1)

In [ ]:
best_model = torch.load('./models/custom_model.pt')

loss_fn = nn.CrossEntropyLoss()

def accuracy(y_predicts, y_labels):
    acc = (y_predicts == y_labels).sum().item() / y_predicts.size(0)
    return acc

best_model.eval()
running_loss_test = []
running_acc_test = []

for batch in tqdm(valid_dataloader):

    inputs, labels = batch
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)

    outputs = best_model(inputs).to(DEVICE)
    y_pred = nn.Softmax(dim=1)(outputs).argmax(dim=1).to(DEVICE)
    loss = loss_fn(outputs, labels)

    running_loss_test.append(loss.item())
    running_acc_test.append(accuracy(y_pred, labels))


print(np.mean(running_loss_test))
print(np.mean(running_acc_test))